In [3]:
import os
os.chdir("..")

In [4]:
import django
# In case that we need it later
# from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE',
'mimuse.settings')
# This is for async, in case we will see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()


In [5]:
import json
import http.client
from songs.models import Song, Album
from artists.models import Artist
from datetime import datetime

In [6]:
Song.objects.all().delete()

(20, {'songs.Song': 20})

In [7]:
import http.client

conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "492a68091cmsh28fa0cbbfc77d53p1a5ad0jsnc64f45e3f32c",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/recommendations/?limit=20&seed_tracks=38MKW2tQHtyO8djIOKlEFF", headers=headers)

res = conn.getresponse()
data = json.loads(res.read().decode("utf-8"))

# print(data.decode("utf-8"))

AttributeError: 'dict' object has no attribute 'decode'

In [8]:
song_list = list()

track = data['tracks']
for song in data['tracks']:
    # print(song['album']['release_date'])
    if(song['album']['release_date_precision']=="day"):
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], release_date= song['album']['release_date'], popularity=song['popularity']))
    else:
        song_list.append(Song(api_id=song['id'], title = song['name'], preview_url= song['preview_url'], popularity=song['popularity']))

    # print (song)
# print(song_list)
bulk = Song.objects.bulk_create(song_list)

In [16]:
# for a in Song.objects.all():
#     a.artist.add(object)
for song in data['tracks']:
    print(song['artists'][0]['name'])
    # if(song['album']['release_date_precision']=="day"):

MAMAMOO
VIVIZ
miss A
(G)I-DLE
IZ*ONE
HWASA
YENA
EVERGLOW
(G)I-DLE
TAEMIN
JEON SOMI
P1Harmony
(G)I-DLE
MISAMO
LOONA
CHUNG HA
K/DA
KWON EUNBI
Billlie
SHINee
